In [1]:
!pip -q install sagemaker sagemaker[local]

You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/hunkim-easy-ocr-data'
role = 'arn:aws:iam::294038372338:role/hunkimSagemaker'

In [3]:
#Make a dummy model.tar.gz
import tarfile
import io

with tarfile.open('model.tar.gz', "w:gz") as tar:
  dummy = b"dummy"
  info = tarfile.TarInfo('dummy.bin')
  info.size = len(dummy)
  tar.addfile(info, io.BytesIO(dummy))

In [4]:
dummy_model = sagemaker_session.upload_data(path='model.tar.gz', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(dummy_model))

'upload_data' method will be deprecated in favor of 'S3Uploader' class (https://sagemaker.readthedocs.io/en/stable/s3.html#sagemaker.s3.S3Uploader) in SageMaker Python SDK v2.
input spec (in this case, just an S3 path): s3://sagemaker-us-west-2-294038372338/sagemaker/hunkim-easy-ocr-data/model.tar.gz


In [5]:
from sagemaker.pytorch import PyTorchModel

# FIXME: Need to upload Dummy model (perhaps only model.targ.gz)
print(dummy_model)
model = PyTorchModel(model_data=dummy_model,
                     role=role,
                     framework_version='1.5.0',
                     entry_point='ocr_infer.py',
                     source_dir='code')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
s3://sagemaker-us-west-2-294038372338/pytorch-training-2020-06-14-12-08-26-889/output/model.tar.gz


In [6]:
%%time
predictor = model.deploy(initial_instance_count=1, instance_type='ml.g4dn.xlarge')

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
---------------!CPU times: user 453 ms, sys: 73.5 ms, total: 527 ms
Wall time: 7min 49s


In [7]:
# Get the end point
endpoint = predictor.endpoint  
print(endpoint)
print("See the logs at", "https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#logEventViewer:group=/aws/sagemaker/Endpoints/"+ endpoint)

pytorch-inference-2020-07-09-09-02-03-598
See the logs at https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#logEventViewer:group=/aws/sagemaker/Endpoints/pytorch-inference-2020-07-09-09-02-03-598


In [8]:
%%time
import boto3
import json

client = boto3.client('sagemaker-runtime')

img = open('code/example.jpg', 'rb').read()

response = client.invoke_endpoint(
    EndpointName=endpoint, 
    ContentType='application/x-image', 
    #Accept="application/json" ,
    Body=bytearray(img)
)

print(response['Body'].read()) 

b'[[[[135, 21], [223, 21], [223, 61], [135, 61]], "\\uae40\\uc9c4\\uc911", 0.9717708826065063], [[[138, 68], [206, 68], [206, 94], [138, 94]], "6 hrs", 0.68975830078125], [[[799, 151], [937, 151], [937, 189], [799, 189]], "OBS\\uc058", 0.18216069042682648], [[[779, 435], [815, 435], [815, 473], [779, 473]], "N", 0.9850679039955139], [[[256, 614], [758, 614], [758, 670], [256, 670]], "\\uc798\\ub9de\\uc9c0 \\uc54a\\ub294 \\uc0ac\\ub78c\\ub4e4 \\ubc11\\uc5d0\\uc11c", 0.11080808937549591], [[[360, 670], [656, 670], [656, 726], [360, 726]], "\\uc77c\\ud558\\uc9c0\\ub294 \\ub9c8\\uc138\\uc694", 0.5123369693756104], [[[274, 800], [344, 800], [344, 824], [274, 824]], "OB5HD", 0.48129624128341675], [[[624, 800], [678, 800], [678, 824], [624, 824]], "OB5}", 0.1305008828639984], [[[964, 800], [1012, 800], [1012, 824], [964, 824]], "OB5", 0.5719876885414124], [[[397, 1029], [623, 1029], [623, 1065], [397, 1065]], "\\uadfc\\ud560 \\ub54c\\uae30\\ubd84\\uc774 \\uc88b\\uc740 N", 0.06049000099301338]

In [9]:
sagemaker_session.delete_endpoint(predictor.endpoint)